In [49]:
import os
import SimpleITK as sitk
from visual import plot_3d, slices_slider
import numpy as np

import matplotlib.pyplot as plt
from skimage import measure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.patches as mpatches
%matplotlib inline

In [2]:
DS_PATH = "/Users/dutking/LOCAL/_datasets/gnatology"

In [3]:
def read_series(path):
    series_reader = sitk.ImageSeriesReader()
    series_IDs = series_reader.GetGDCMSeriesIDs(path)
    if not series_IDs:
        print(f"ERROR: given directory {path} does not contain a DICOM series.")
        return
    series_file_names = series_reader.GetGDCMSeriesFileNames(path, series_IDs[0])
    series_reader.SetFileNames(series_file_names)
    image3D = series_reader.Execute()
    return image3D

In [46]:
img = read_series(os.path.join(DS_PATH, "1"))
print(img.GetSize(), img.GetSpacing(), img.GetOrigin())
img = sitk.Cast(img[430:, 430:, 500:540], sitk.sitkFloat32)
# img_arr = sitk.GetArrayFromImage(img)
# img_arr_part = img_arr[500:540, 430:, 430:]

(860, 860, 760) (0.2, 0.2, 0.2) (0.0, 0.0, 0.2)


In [57]:
lower_threshold = 150  # Adjust as needed
upper_threshold = 1000  # Adjust as needed
# smooth_img = sitk.SmoothingRecursiveGaussian(img, sigma=0.5)
# edge_image = sitk.CannyEdgeDetection(
#     img,
#     lowerThreshold=1000,
#     upperThreshold=1500,
#     variance=[1, 1, 1],
# )


bone_mask = sitk.BinaryThreshold(
    img, lower_threshold, upper_threshold, insideValue=1, outsideValue=0
)
bone_mask = sitk.BinaryDilate(bone_mask, (3, 3, 3))
bone_mask = sitk.BinaryErode(bone_mask, (3, 3, 3))
segmented_bones = sitk.Mask(img, bone_mask)
segmented_bones_arr = sitk.GetArrayFromImage(segmented_bones)
slices_slider(segmented_bones_arr, 20, "img")

interactive(children=(IntSlider(value=20, description='axial_slice', max=39), Output()), _dom_classes=('widget…